### Requirements
- matplotlib
- numpy
- pandas
- seaborn
- scipy
- requests
- (optional) tqdm

In [ ]:
import pathlib
import sys
PATH = pathlib.Path().absolute().resolve()

%matplotlib inline
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import matplotlib.colors as clr
import seaborn as sns
    
sns.set(style="whitegrid", font_scale=1.4)
COLORS = sns.color_palette()

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
import json
import requests
try:
    from tqdm import tqdm
except ImportError:
    # tqdm placeholder
    class tqdm:
        def __init__(self, it=None): 
            self.it=it
        def __enter__(self): 
            return self
        def __exit__(self, *args, **kwargs): 
            pass
        def __iter__(self): 
            return iter(self.it)
        def update(*args): 
            pass

In [ ]:
leaderbord_url = lambda offset, limit=100: f"https://api.2018.halite.io/v1/api/leaderboard?offset={offset}&limit={limit}"

In [ ]:
def ax_style(ax):
    ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
    ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
    ax.legend()
    ax.minorticks_on()
    return ax

In [ ]:
leaderboard_data = []
chunk = True
step = 100
i = 0
with tqdm() as progress:
    while chunk:
        url = leaderbord_url(step * i, step)
        i += 1
        chunk = requests.get(url).json()
        leaderboard_data.extend(chunk)
        progress.update(step)

In [ ]:
MU_CUT = 30
SIGMA_CUT = 2
df = pd.DataFrame(leaderboard_data)
df = df[df.mu >= MU_CUT]
df_filtered = df[df.sigma <= SIGMA_CUT]

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
df_filtered.plot(x="rank", y="mu", ax=ax, marker=',', linestyle=' ', label=r"$\mu$ raw")
Y_smooth = savgol_filter(df.mu, 101, 3)
ax.plot(df["rank"], Y_smooth, label=r"$\mu$")

ax.set_xlabel("Ladder position")
ax.set_ylabel(r"$\mu$")
ax.set_xlim(len(df), 0)
ax_style(ax)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
for i, (df_, ax) in enumerate(((df, ax1), (df[df["rank"] <= 100], ax2))):
    values = df_.language.value_counts()[::-1]
    values.plot(ax=ax, kind='barh', logx=not i)
    ax_style(ax)
    ax.set_xlim(not i, values.max() * 1.1)
ax1.set_title("All players")
ax2.set_title("Top100 players")
fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 30))
ax1 = plt.subplot2grid((3, 1), (0, 0))
ax2 = plt.subplot2grid((3, 1), (1, 0))
rank_cut = 200
top_100 = {}
for lang in df.language.unique():
    df_ = df[df.language == lang]
    if len(df_) >= 30:
        sns.distplot(df_.score, ax=ax1, hist=False, label=lang)
        sns.distplot(df_["rank"], ax=ax2, hist=False, bins=20, label=lang)
        
    values = df_[df_["rank"] < rank_cut]["rank"]
    if len(values) >= 3:
        top_100[lang] = values
        
sns.distplot(df[df.is_gpu_enabled == True]["rank"], ax=ax2, kde_kws={"ls": "--"}, hist=False, label="with GPU")
        
top_100 = sorted(top_100.items(), reverse=True,
                 key=lambda k_v: df[(df.language == k_v[0]) & (df["rank"] <= rank_cut)].score.mean())
top_100_k, top_100_v = zip(*top_100)

ax_style(ax1)
ax1.set_xlim(df.score.max(), MU_CUT)
ax1.set_xlabel(r"Score = $\mu - 3\sigma$")
ax1.set_ylabel("Distribution")
ax1.set_title("Distrubution of score by prorg. lang.")

ax_style(ax2)
ax2.set_xlim(1, df["rank"].max())
ax2.set_xlabel(r"Rank")
ax2.set_ylabel("Distribution")
ax2.set_title("Distrubution of rank by prorg. lang.")

with sns.axes_style(style="darkgrid"):
    ax3 = plt.subplot2grid((3, 1), (2, 0))
    ax3.hist(
        top_100_v, 
        bins=np.arange(0, rank_cut + rank_cut // 10, 10), 
        stacked=True, label=top_100_k,
        color=sns.color_palette("Spectral", len(top_100_k)),
        zorder=-1
    )
    
    ax_style(ax3)
    ax3.set_xlim(1, rank_cut + 40)
    ax3.set_xlabel(f"Rank (top{rank_cut})")
    ax3.set_ylabel("Distribution")
    ax3.set_title(f"Distrubution of rank (top{rank_cut}) by prorg. lang.")
        

fig.tight_layout()
plt.show()